In [23]:
import random
import time

# 函数用于使用Miller-Rabin测试检查n是否为质数
def miller_rabin_test(n, k=50):
    """运行Miller-Rabin素性测试k次来检查n是否可能是质数。"""
    if n == 2 or n == 3:
        return True  # 2和3是质数
    if n <= 1 or n % 2 == 0:
        return False  # 小于等于1或偶数不是质数

    # 将n写成d * 2^r + 1的形式，其中d为奇数
    r, d = 0, n - 1
    while d % 2 == 0:
        r += 1
        d //= 2  # 不断除以2直到d变为奇数

    # 进行Miller-Rabin测试的多轮检测
    for _ in range(k):
        a = random.randint(2, n - 2)  # 随机选择一个基础a
        x = pow(a, d, n)  # 计算a^d mod n
        if x == 1 or x == n - 1:
            continue  # 如果x等于1或n-1，则此轮测试通过

        for _ in range(r - 1):
            x = pow(x, 2, n)  # 计算x^2 mod n
            if x == n - 1:
                break  # 如果x等于n-1，则此轮测试通过
        else:
            return False  # 如果所有循环都没有找到x=n-1，则n不是质数
    return True  # 所有测试都通过了，n很可能是质数

# 函数用于在范围[2^1023, 2^1024]内生成随机质数
def generate_large_prime():
    """在范围[2^1023, 2^1024]内生成随机质数。"""
    while True:
        candidate = random.getrandbits(1024)  # 生成一个随机的1024位数字
        candidate |= (1 << 1023) | 1  # 确保该数字在范围[2^1023, 2^1024]内并且是奇数
        if miller_rabin_test(candidate):  # 使用Miller-Rabin测试验证候选数
            return candidate  # 返回通过测试的候选数作为质数

# 函数记录生成质数所需的时间
def timed_prime_generation():
    start_time = time.time()  # 记录开始时间
    prime = generate_large_prime()  # 调用函数生成大质数
    end_time = time.time()  # 记录结束时间
    time_taken = end_time - start_time  # 计算所需时间
    return prime, time_taken  # 返回生成的质数和所花费的时间

# 示例使用
prime_number, computation_time = timed_prime_generation()  # 调用带计时功能的质数生成函数
print(f"生成的质数: {prime_number}")  # 输出生成的质数
print(f"耗时: {computation_time:.2f} 秒")  # 输出计算耗时

生成的质数: 123284514247091755911574275345552673883515350352765040419960490164924850282819652476448420981025534610840388804724734927945693642567515718291935427736851452303433812127326298163710826225042883187921541198857626962813721994036617485805161604762274044511360854068403855080932271230169167321368915001439015667773
耗时: 0.64 秒


In [28]:
import math
import random
import time

# Pollard's Rho算法用于整数分解
def pollards_rho(n):
    """使用Pollard's Rho算法返回n的一个非平凡因子。"""
    if n % 2 == 0:
        return 2
    x = random.randint(2, n - 1)
    y = x
    c = random.randint(1, n - 1)
    d = 1

    # 迭代使用的函数
    def f(x):
        return (x * x + c) % n

    while d == 1:
        x = f(x)
        y = f(f(y))
        d = math.gcd(abs(x - y), n)
        if d == n:  # 失败，尝试使用新的随机参数再次执行
            return pollards_rho(n)
    return d

# 使用Pollard's Rho反复分解给定的数字
def factorize(n):
    """使用Pollard's Rho分解n并返回其质因数列表。"""
    factors = []
    # 对小因子进行试除法
    for i in range(2, 1000):
        while n % i == 0:
            factors.append(i)
            n //= i
        if n == 1:
            return factors

    # 使用Pollard's Rho对较大因子进行处理
    while n > 1:
        if is_prime(n):
            factors.append(n)
            break
        factor = pollards_rho(n)
        while n % factor == 0:
            factors.append(factor)
            n //= factor
    return factors

# 较小数字的基本素性测试
def is_prime(n):
    """检查n是否为质数。"""
    if n <= 1:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    for i in range(3, int(math.sqrt(n)) + 1, 2):
        if n % i == 0:
            return False
    return True

# 记录分解数字所需的时间
def timed_factorization(n):
    start_time = time.time()
    factors = factorize(n)
    end_time = time.time()
    time_taken = end_time - start_time
    return factors, time_taken

n = [12345678451,525344836890429,4738299074862866237,49273482537578568628373] 
for i in n:
    factors, computation_time = timed_factorization(i)
    print(f"Factors of {i}: {factors}")
    print(f"Time taken: {computation_time:.8f} seconds")



Factors of 12345678451: [2473, 4992187]
Time taken: 0.00000000 seconds
Factors of 525344836890429: [3, 3, 3, 19, 7351, 139309483]
Time taken: 0.00000000 seconds
Factors of 4738299074862866237: [18311, 258767903165467]
Time taken: 0.35912657 seconds
Factors of 49273482537578568628373: [7, 7, 48989, 2267, 130681, 69287459]
Time taken: 0.00384188 seconds
